[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BHKLAB-DataProcessing/readii-idc-notebooks/blob/jjjermiah-dev/notebooks/1_RunPyradiomics.ipynb)

In [1]:
import logging
from collections import Counter
from pathlib import Path

import pandas as pd
from readii.feature_extraction import singleRadiomicFeatureExtraction
from rich import print

from readii_idc_notebooks import generate_image_mask_pairs

pyradiomics_logger = logging.getLogger("radiomics")
pyradiomics_logger.setLevel(logging.ERROR)

# Setup and Configuration

In [2]:
# a Save data to local directory
DATA_DIR = Path("data")

# If you choose a different collection in the setup notebook, you will need to change this value
COLLECTION_ID = "nsclc_radiomics"

NIFTI_OUTPUT_DIR = DATA_DIR / "images" / COLLECTION_ID / "niftis"

PYRADIOMICS_CONFIG = Path().cwd().parent / "pyradiomics.yaml"
assert PYRADIOMICS_CONFIG.exists()

RANDOM_SEED = 42

! tree -F $NIFTI_OUTPUT_DIR

data/images/nsclc_radiomics/niftis
├── dataset_index.csv
├── SubjectID-0_LUNG1-254/
│   ├── CT_SeriesUID-65670/
│   │   ├── original.nii.gz
│   │   ├── randomized_full.nii.gz
│   │   ├── randomized_non_roi.nii.gz
│   │   ├── randomized_roi.nii.gz
│   │   ├── randomized_sampled_full.nii.gz
│   │   ├── randomized_sampled_non_roi.nii.gz
│   │   ├── randomized_sampled_roi.nii.gz
│   │   ├── shuffled_full.nii.gz
│   │   ├── shuffled_non_roi.nii.gz
│   │   └── shuffled_roi.nii.gz
│   └── RTSTRUCT_SeriesUID-44321/
│       └── GTV.nii.gz
├── SubjectID-1_LUNG1-289/
│   ├── CT_SeriesUID-98684/
│   │   ├── original.nii.gz
│   │   ├── randomized_full.nii.gz
│   │   ├── randomized_non_roi.nii.gz
│   │   ├── randomized_roi.nii.gz
│   │   ├── randomized_sampled_full.nii.gz
│   │   ├── randomized_sampled_non_roi.nii.gz
│   │   ├── randomized_sampled_roi.nii.gz
│   │   ├── shuffled_full.nii.gz
│   │   ├── shuffled_non_roi.nii.gz
│   │   └── shuffled_roi.nii.gz
│   └── RTSTRUCT_SeriesUID-86747/
│       

In [3]:
results = []
# Example usage:
for i, subject in enumerate(generate_image_mask_pairs(NIFTI_OUTPUT_DIR)):
    metadata = {
                "patient_ID": subject.PatientID,
                "image_ID": subject.image_id,
                "mask_ID": subject.mask_id,
                "image_modality": subject.image_modality,
                "mask_modality": subject.mask_modality,
            }
    feature_vector = singleRadiomicFeatureExtraction(
            ctImage=subject.image,
            roiImage=subject.mask,
            pyradiomicsParamFilePath=PYRADIOMICS_CONFIG,
            randomSeed=RANDOM_SEED,
        )
    metadata.update(feature_vector)
    results.append(
        metadata
    )
pyradiomic_features = pd.DataFrame(results)

Processing 4_LUNG1-241 : original                      : 100%|██████████| 50/50 [12:14<00:00, 14.69s/it]  


In [9]:
column_type_counts = Counter(
  pyradiomic_features.columns[5:].map(lambda cname: cname.split("_")[0])
)

summary = {
    "total_patients" : pyradiomic_features["patient_ID"].nunique(),
    "total_rows": len(pyradiomic_features),
    "image_types" : sorted(list(pyradiomic_features["image_ID"].unique())),
    "column_type_counts": column_type_counts,
}
print("Summary:", summary)

pyradiomic_features.to_csv(DATA_DIR / "pyradiomic_features.csv", index=False)

Summary:
{
    'total_patients': 5,
    'total_rows': 50,
    'image_types': [
        'original',
        'randomized_full',
        'randomized_non_roi',
        'randomized_roi',
        'randomized_sampled_full',
        'randomized_sampled_non_roi',
        'randomized_sampled_roi',
        'shuffled_full',
        'shuffled_non_roi',
        'shuffled_roi'
    ],
    'column_type_counts': Counter({
        'diagnostics': 37,
        'original': 107,
        'wavelet-LLH': 93,
        'wavelet-LHL': 93,
        'wavelet-LHH': 93,
        'wavelet-HLL': 93,
        'wavelet-HLH': 93,
        'wavelet-HHL': 93,
        'wavelet-HHH': 93,
        'wavelet-LLL': 93,
        'square': 93,
        'squareroot': 93,
        'logarithm': 93,
        'exponential': 93,
        'gradient': 93
    })
}